In [2]:
from llama_index.evaluation import generate_question_context_pairs
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.node_parser import SentenceSplitter
from llama_index.llms import OpenAI

In [4]:
documents = SimpleDirectoryReader("./data").load_data()

In [5]:
documents

[Document(id_='7cbfea24-9845-420d-b1e9-228cff584434', embedding=None, metadata={'file_path': 'data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2023-12-25', 'last_modified_date': '2023-12-25', 'last_accessed_date': '2023-12-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='363f3787955dd09439335327f171239ffa7e63a427910a448ced198ba55a8e30', text='\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with 

In [6]:
node_parser = SentenceSplitter(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)

In [8]:
# by default, the node ids are set to random uuids. To ensure same id's per run, we manually set them.
for idx, node in enumerate(nodes):
    node.id_ = f"node_{idx}"

In [9]:
len(nodes)

57

In [14]:
nodes[0].__dict__

{'id_': 'node_0',
 'embedding': None,
 'metadata': {'file_path': 'data/paul_graham_essay.txt',
  'file_name': 'paul_graham_essay.txt',
  'file_type': 'text/plain',
  'file_size': 75042,
  'creation_date': '2023-12-25',
  'last_modified_date': '2023-12-25',
  'last_accessed_date': '2023-12-25'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7cbfea24-9845-420d-b1e9-228cff584434', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2023-12-25', 'last_modified_date': '2023-12-25', 'last_accessed_date': '2023-12-25'}, hash='363f3787955dd09

In [54]:
import os
# os.environ["OPENAI_API_KEY"] = ""

llm = OpenAI(model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)

In [20]:
vector_index = VectorStoreIndex(nodes, service_context=service_context)
retriever = vector_index.as_retriever(similarity_top_k=2)

In [21]:
retrieved_nodes = retriever.retrieve("What did the author do growing up?")

In [23]:
from llama_index.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** node_0<br>**Similarity:** 0.8173553751591254<br>**Text:** What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in ...<br>

**Node ID:** node_34<br>**Similarity:** 0.8134730762483309<br>**Text:** Except for a few officially anointed thinkers who went to the right parties in New York, the only people allowed to publish essays were specialists writing about their specialties. There were so many essays that had never been written, because there had been no way to publish them. Now they could be, and I was going to write them. [12]

I've worked on several different things, but to the extent there was a turning point where I figured out what to work on, it was when I started publishing essays online. From then on I knew that whatever else I did, I'd always write essays too.

I knew that online essays would be a marginal medium at first. Socially they'd seem more like rants posted by nutjobs on their GeoCities sites than the genteel and beautifully typeset compositions published in The New Yorker. But by this point I knew enough to find that encouraging instead of discouraging.

One of the most conspicuous patterns I've noticed in my life is how well it has worked, for me at least...<br>

In [24]:
# build evaluation dataset
from llama_index.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)

qa_dataset = generate_question_context_pairs(
    nodes, llm=llm, num_questions_per_chunk=2
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [04:34<00:00,  4.82s/it]


In [25]:
queries = qa_dataset.queries.values()
print(list(queries)[2])

"Describe the transition from using punched cards as a form of input to programs to the advent of microcomputers, as discussed in the text. How did this change impact the author's experience with programming?"


In [26]:
# save and load data
qa_dataset.save_json("pg_eval_dataset.json")
# qa_dataset = EmbeddingQAFinetuneDataset.from_json("pg_eval_dataset.json")

In [27]:
from llama_index.evaluation import RetrieverEvaluator

metrics = ["mrr", "hit_rate"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)

In [36]:
import asyncio

# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]
print(sample_id, sample_query)

eval_result = await retriever_evaluator.aevaluate(sample_query, sample_expected)
print(eval_result)

099e61fb-66b8-49b2-8925-f3f168d61fc2 "Describe the author's initial experiences with programming on the IBM 1401. What were some of the limitations he faced and how did this impact his ability to create meaningful programs?"
Query: "Describe the author's initial experiences with programming on the IBM 1401. What were some of the limitations he faced and how did this impact his ability to create meaningful programs?"
Metrics: {'mrr': 1.0, 'hit_rate': 1.0}



In [37]:
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [38]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()
    columns = {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}

    metric_df = pd.DataFrame(columns)

    return metric_df

In [39]:
display_results("top-2 eval", eval_results)

retrievers  hit_rate       mrr
0  top-2 eval   0.77193  0.657895

In [46]:
from llama_index.schema import TextNode

my_node = TextNode(text='2023/9/8 21:45 OPPO宣布 Find N3 Flip 今⽇全⾯开售https://md2pdf.netlify .app 1/2OPPO宣布 Find N3 Flip 今⽇全⾯开售【环球⽹科技综合报道】 9 ⽉ 8 ⽇消息， OPPO 宣布其全新⼀代  Find N3 Flip 于 9 ⽉ 8 ⽇ 10 点全⾯开售。即⽇起在OPPO 商城、授权电商、 OPPO 体验店、授权零售商⻔店等线上线下全渠道购机的⽤户，有机')

In [47]:
my_node.__dict__

{'id_': 'a0294763-067f-4e99-ab3d-ba584afce762',
 'embedding': None,
 'metadata': {},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'hash': 'de104317b32ac006e7258c4a3b3303fa697e2a240a96ced6addb5c4df175d430',
 'text': '2023/9/8 21:45 OPPO宣布 Find N3 Flip 今⽇全⾯开售https://md2pdf.netlify .app 1/2OPPO宣布 Find N3 Flip 今⽇全⾯开售【环球⽹科技综合报道】 9 ⽉ 8 ⽇消息， OPPO 宣布其全新⼀代  Find N3 Flip 于 9 ⽉ 8 ⽇ 10 点全⾯开售。即⽇起在OPPO 商城、授权电商、 OPPO 体验店、授权零售商⻔店等线上线下全渠道购机的⽤户，有机',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n'}

In [51]:
# Prompt to generate questions
qa_generate_prompt_tmpl = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination using Chinese. The questions should be diverse in nature \
across the document. The questions should not contain options, not start with Q1/ Q2. \
Restrict the questions to the context information provided.
"""
my_qa_dataset = generate_question_context_pairs(
    [my_node], llm=llm, num_questions_per_chunk=2, qa_generate_prompt_tmpl=qa_generate_prompt_tmpl
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.85s/it]


In [52]:
my_qa_dataset

EmbeddingQAFinetuneDataset(queries={'d98e71c9-386a-44dd-b244-f74a74e6e642': '请解释OPPO Find N3 Flip的发布日期和销售渠道。', '6c19835c-e5ac-4bf6-891a-87503a54c177': 'OPPO Find N3 Flip在哪些线上线下渠道开始全面销售？'}, corpus={'a0294763-067f-4e99-ab3d-ba584afce762': '2023/9/8 21:45 OPPO宣布 Find N3 Flip 今⽇全⾯开售https://md2pdf.netlify .app 1/2OPPO宣布 Find N3 Flip 今⽇全⾯开售【环球⽹科技综合报道】 9 ⽉ 8 ⽇消息， OPPO 宣布其全新⼀代  Find N3 Flip 于 9 ⽉ 8 ⽇ 10 点全⾯开售。即⽇起在OPPO 商城、授权电商、 OPPO 体验店、授权零售商⻔店等线上线下全渠道购机的⽤户，有机'}, relevant_docs={'d98e71c9-386a-44dd-b244-f74a74e6e642': ['a0294763-067f-4e99-ab3d-ba584afce762'], '6c19835c-e5ac-4bf6-891a-87503a54c177': ['a0294763-067f-4e99-ab3d-ba584afce762']}, mode='text')

In [53]:
my_qa_dataset.queries

{'d98e71c9-386a-44dd-b244-f74a74e6e642': '请解释OPPO Find N3 Flip的发布日期和销售渠道。',
 '6c19835c-e5ac-4bf6-891a-87503a54c177': 'OPPO Find N3 Flip在哪些线上线下渠道开始全面销售？'}